# Learning: Control Algorithms

This notebook teaches you about different control algorithms and how to implement them. **You'll need to fill in the missing code!**

## What You'll Learn

1. Understanding different control approaches
2. Implementing a Proportional (P) controller
3. Implementing a Proportional-Integral-Derivative (PID) controller
4. Implementing a Model Predictive Control (MPC) approach
5. Comparing controller performance

## Instructions

- Read each section carefully
- Look for `# TODO:` comments - these indicate where you need to write code
- Fill in the `...` placeholders
- Run cells as you complete them to test your work

## Prerequisites

- Understanding of controllers (see [Controller Tutorial](../tutorials/controller_tutorial.ipynb))
- Basic understanding of control theory (helpful but not required)
- Understanding of classes (see [Understanding Classes and OOP](../oop_fundamentals/understanding_classes_and_oop.ipynb))

In [ ]:
import sys

sys.path.insert(0, '../../../src')

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    BaseController,
    Car,
    CarState,
    Track,
    TrackPlanner,
)

## Step 1: Proportional (P) Controller

A **Proportional controller** adjusts the output proportionally to the error:

```
output = Kp * error
```

Where:
- **Kp**: Proportional gain
- **error**: Difference between desired and actual value

**TODO**: Implement a simple P controller for heading control!

In [ ]:
class PController(BaseController):
    """
    Proportional controller for path following.

    TODO: Fill in the implementation!
    """

    def __init__(self, kp_heading=2.0, kp_velocity=0.5, target_velocity=10.0, name="p_controller"):
        """
        Initialize P controller.

        TODO: Store gains and target velocity
        """
        super().__init__(name=name)
        # TODO: Store kp_heading, kp_velocity, target_velocity
        ...

    def compute_control(self, car_state, perception_data=None, costmap=None, plan=None, dt=0.1):
        """
        Compute control using P controller.

        TODO:
        1. If no plan, return zero control
        2. Find closest waypoint in plan to car position
        3. Calculate desired heading (direction to waypoint)
        4. Calculate heading error (desired - current)
        5. Normalize heading error to [-pi, pi]
        6. Calculate steering_rate = kp_heading * heading_error
        7. Calculate velocity error = target_velocity - current_velocity
        8. Calculate acceleration = kp_velocity * velocity_error
        9. Return control dict
        """
        if plan is None or len(plan) == 0:
            return {"acceleration": 0.0, "steering_rate": 0.0}

        # TODO: Find closest waypoint
        car_state.position()
        closest_idx = ...  # Find index of closest waypoint
        plan[closest_idx]

        # TODO: Calculate heading error
        heading_error = ...  # desired - current
        heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))  # Normalize

        # TODO: Calculate control outputs
        steering_rate = ...  # kp_heading * heading_error

        acceleration = ...  # kp_velocity * velocity_error

        return {
            "acceleration": acceleration,
            "steering_rate": steering_rate
        }

# Test the controller
p_controller = PController(kp_heading=2.0, kp_velocity=0.5, target_velocity=10.0)

car = Car(initial_state=CarState(x=0, y=0, heading=0, velocity=8.0))
track = Track.create_simple_track(length=100.0, width=50.0, track_width=5.0)
planner = TrackPlanner(track)
plan = planner.plan(car.state)

control = p_controller.compute_control(car.state, plan=plan)
print(f"P Controller output: {control}")

## Step 2: PID Controller

A **PID controller** adds Integral and Derivative terms to the Proportional term:

```
output = Kp * error + Ki * integral + Kd * derivative
```

Where:
- **Kp**: Proportional gain
- **Ki**: Integral gain (eliminates steady-state error)
- **Kd**: Derivative gain (reduces overshoot)

**TODO**: Implement a PID controller with integral and derivative terms!

In [ ]:
class PIDControllerCustom(BaseController):
    """
    PID controller for path following.

    TODO: Fill in the implementation!
    """

    def __init__(self, kp=1.0, ki=0.1, kd=0.01, target_velocity=10.0, name="pid_controller"):
        """
        Initialize PID controller.

        TODO: Store gains, target velocity, and initialize error history
        """
        super().__init__(name=name)
        # TODO: Store kp, ki, kd, target_velocity
        ...

        # TODO: Initialize error history for integral and derivative
        self.heading_error_integral = 0.0
        self.heading_error_previous = 0.0

    def compute_control(self, car_state, perception_data=None, costmap=None, plan=None, dt=0.1):
        """
        Compute control using PID controller.

        TODO:
        1. Calculate heading error (same as P controller)
        2. Calculate integral: heading_error_integral += heading_error * dt
        3. Calculate derivative: (heading_error - heading_error_previous) / dt
        4. Calculate steering_rate = kp*error + ki*integral + kd*derivative
        5. Update heading_error_previous for next iteration
        6. Do similar for velocity control
        """
        if plan is None or len(plan) == 0:
            return {"acceleration": 0.0, "steering_rate": 0.0}

        # TODO: Calculate heading error (same as P controller)
        car_pos = car_state.position()
        distances = np.linalg.norm(plan - car_pos, axis=1)
        closest_idx = np.argmin(distances)
        target = plan[closest_idx]

        direction = target - car_pos
        desired_heading = np.arctan2(direction[1], direction[0])
        heading_error = desired_heading - car_state.heading
        heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))

        # TODO: Calculate PID terms
        # Proportional term

        # Integral term
        self.heading_error_integral += ...  # heading_error * dt

        # Derivative term

        # TODO: Calculate steering rate
        steering_rate = ...  # p_term + i_term + d_term

        # Update previous error
        self.heading_error_previous = ...

        # TODO: Similar for velocity (simpler - just P term is fine)
        acceleration = ...  # kp_velocity * velocity_error (use kp for simplicity)

        return {
            "acceleration": acceleration,
            "steering_rate": steering_rate
        }

# Test the controller
pid_controller = PIDControllerCustom(kp=1.5, ki=0.2, kd=0.05, target_velocity=10.0)

control = pid_controller.compute_control(car.state, plan=plan)
print(f"PID Controller output: {control}")

## Step 3: Model Predictive Control (MPC) Approach

**Model Predictive Control** predicts future states and optimizes control over a horizon.

For this exercise, we'll implement a simplified MPC that:
1. Predicts future positions using current control
2. Evaluates cost of different control sequences
3. Selects the best control

**TODO**: Implement a simple MPC controller!

In [ ]:
class SimpleMPCController(BaseController):
    """
    Simplified Model Predictive Control for path following.

    TODO: Fill in the implementation!
    """

    def __init__(self, horizon=5, target_velocity=10.0, name="mpc_controller"):
        """
        Initialize MPC controller.

        TODO: Store horizon and target velocity
        """
        super().__init__(name=name)
        # TODO: Store horizon, target_velocity
        ...

    def _predict_state(self, car_state, steering_rate, dt=0.1, steps=1):
        """
        Predict future car state.

        TODO: Simple bicycle model prediction
        """
        # Simplified: assume constant velocity, update heading
        predicted_state = CarState(
            x=car_state.x,
            y=car_state.y,
            heading=car_state.heading + steering_rate * dt * steps,
            velocity=car_state.velocity
        )

        # TODO: Update position based on heading and velocity
        predicted_state.x += ...  # velocity * cos(heading) * dt * steps
        predicted_state.y += ...  # velocity * sin(heading) * dt * steps

        return predicted_state

    def _evaluate_cost(self, predicted_state, plan):
        """
        Evaluate cost of predicted state.

        TODO: Calculate distance to nearest waypoint in plan
        """
        if len(plan) == 0:
            return 1000.0  # High cost if no plan

        # TODO: Find distance to nearest waypoint
        predicted_state.position()
        min_distance = ...  # Minimum distance

        return min_distance

    def compute_control(self, car_state, perception_data=None, costmap=None, plan=None, dt=0.1):
        """
        Compute control using MPC.

        TODO:
        1. Generate candidate steering rates (e.g., -1.0 to 1.0 rad/s)
        2. For each candidate:
           - Predict state over horizon
           - Evaluate cost
        3. Select candidate with minimum cost
        4. Return control
        """
        if plan is None or len(plan) == 0:
            return {"acceleration": 0.0, "steering_rate": 0.0}

        # TODO: Generate candidate steering rates
        candidates = np.linspace(-1.0, 1.0, 11)  # 11 candidates from -1 to 1 rad/s

        best_cost = float('inf')
        best_steering_rate = 0.0

        # TODO: Evaluate each candidate
        for steering_rate in candidates:
            # Predict state over horizon
            predicted_state = car_state
            total_cost = 0.0

            for step in range(self.horizon):
                predicted_state = self._predict_state(predicted_state, steering_rate, dt, 1)
                cost = self._evaluate_cost(predicted_state, plan)
                total_cost += cost

            # TODO: Update best if this is better
            if total_cost < best_cost:
                best_cost = ...
                best_steering_rate = ...

        # Simple velocity control
        velocity_error = self.target_velocity - car_state.velocity
        acceleration = 0.5 * velocity_error

        return {
            "acceleration": acceleration,
            "steering_rate": best_steering_rate
        }

# Test the controller
mpc_controller = SimpleMPCController(horizon=5, target_velocity=10.0)

control = mpc_controller.compute_control(car.state, plan=plan)
print(f"MPC Controller output: {control}")

## Step 4: Compare Controllers

Now let's compare the performance of different controllers!

In [ ]:
def simulate_controller(controller, car, planner, track, num_steps=50):
    """Simulate a controller and return trajectory."""
    trajectory = [car.state.position().copy()]

    for _ in range(num_steps):
        plan = planner.plan(car.state)
        control = controller.compute_control(car.state, plan=plan)

        # Update car state (simplified)
        car.state.velocity += control["acceleration"] * 0.1
        car.state.steering_angle += control["steering_rate"] * 0.1
        car.state.heading += control["steering_rate"] * 0.1

        # Update position
        car.state.x += car.state.velocity * np.cos(car.state.heading) * 0.1
        car.state.y += car.state.velocity * np.sin(car.state.heading) * 0.1

        trajectory.append(car.state.position().copy())

    return np.array(trajectory)

# Compare controllers
controllers = [
    (PController(kp_heading=2.0, name="P"), "P Controller"),
    (PIDControllerCustom(kp=1.5, ki=0.2, kd=0.05, name="PID"), "PID Controller"),
    (SimpleMPCController(horizon=5, name="MPC"), "MPC Controller"),
]

fig, axes = plt.subplots(1, len(controllers), figsize=(15, 5))

for idx, (controller, name) in enumerate(controllers):
    # Reset car for each controller
    test_car = Car(initial_state=CarState(x=0, y=0, heading=0, velocity=10.0))

    # Simulate
    trajectory = simulate_controller(controller, test_car, planner, track)

    # Plot
    ax = axes[idx] if len(controllers) > 1 else axes
    track.visualize(ax=ax, frame="global")
    planner.visualize(ax=ax, car_state=test_car.state, plan=planner.plan(test_car.state), frame="global")
    ax.plot(trajectory[:, 0], trajectory[:, 1], 'r-', linewidth=2, label='Trajectory')
    ax.set_title(name)
    ax.legend()

plt.tight_layout()
plt.show()

print("\nComparison complete! Notice the differences in:")
print("  - Path smoothness")
print("  - Convergence to desired path")
print("  - Overshoot/oscillation")

## Summary

You've learned about different control algorithms:

1. **P Controller**: Simple, fast, but may have steady-state error
2. **PID Controller**: More accurate, eliminates steady-state error, but can overshoot
3. **MPC Controller**: Optimizes over a horizon, can handle constraints, but more computationally expensive

### Key Takeaways

- **Proportional (P)**: Reacts to current error
- **Integral (I)**: Eliminates steady-state error
- **Derivative (D)**: Reduces overshoot and oscillation
- **MPC**: Predicts and optimizes future behavior

### Next Steps

- Try tuning the gains (Kp, Ki, Kd) to see how they affect performance
- Experiment with different horizons for MPC
- Compare with the built-in PurePursuitController
- See [Building a Custom Controller](../control/learning_build_controller.ipynb) for more examples